In [11]:
from components.importacao_diretorios_windows import *
from components.extract_text_pdf import extract_text_pdf
from components.importacao_caixa_dialogo import DialogBox
from components.checar_ativacao_google_drive import checa_google_drive
from components.configuracao_db import configura_db, ler_sql
from components.procura_cliente import procura_cliente
from components.procura_valores import procura_valores, procura_valores_com_codigo
from components.procura_elementos_web import procura_elemento, procura_todos_elementos, encontrar_elemento_shadow_root
from components.configuracao_selenium_drive import configura_selenium_driver
from components.enviar_emails import enviar_email_com_anexos
import tkinter as tk
import mysql.connector
from re import search
from pathlib import Path
from shutil import copy
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, NamedStyle
import win32com.client as win32
from dotenv import load_dotenv
import os
from time import sleep, time
from datetime import date
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import  NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
load_dotenv()

# =====================CONFIGURAÇÂO DO BANCO DE DADOS======================
db_conf = configura_db()

# =============CHECANDO SE O GOOGLE FILE STREAM ESTÁ INICIADO NO SISTEMA==============
checa_google_drive()

chrome_options, servico = configura_selenium_driver()

# ================CONFIGURAÇÃO DAS VARIAVEIS DE AMBIENTE=====================
automacao_email = os.getenv('SELENIUM_USER')
automacao_senha = os.getenv('SELENIUM_PASSWORD')
email_gestor = os.getenv('EMAIL_GESTOR')
corpo_email = os.getenv('CORPO_EMAIL')

# ==================CAIXA DE DIALOGO INICIAL============================
def main():
    root = tk.Tk()
    app = DialogBox(root)
    root.mainloop()
    return app.particao, app.rotina, app.mes, app.ano

In [13]:
if __name__ == "__main__":
    particao, rotina, mes, ano = main()

In [88]:
dir_clientes_itaperuna = f"{particao}:\\Meu Drive\\Cobranca_Clientes_terceirizacao\\Clientes Itaperuna"
dir_clientes_manaus = f"{particao}:\\Meu Drive\\Cobranca_Clientes_terceirizacao\\Clientes Manaus"
lista_dir_clientes = [dir_clientes_itaperuna, dir_clientes_manaus]
dir_extratos = f"{particao}:\\Meu Drive\\Robo_Emissao_Relatorios_do_Mes\\faturas_human_{mes}_{ano}"
modelo_fatura = Path(f"{particao}:\\Meu Drive\\Arquivos_Automacao\\Fatura_Detalhada_Modelo_00.0000_python.xlsx")
planilha_vales_sst = Path(f"{particao}:\\Meu Drive\\Relatorio_Vales_Saude_Seguranca\\{mes}-{ano}\\Relatorio_Vales_Saude_Seguranca_{mes}.{ano}.xlsx")
planilha_reembolsos = Path(f"{particao}:\\Meu Drive\\Relatorio_Boletos_Salario_Reembolso\\{mes}-{ano}\\Relatorio_Boletos_Salario_Reembolso.xlsx")
categoria_cobranca = "Gestão de Mão de Obra Terceirizada"

In [114]:
def agendar_lancamento(driver, valor_fatura, actions):
    print("AGENDANDO LANÇAMENTO")
    try:
        elemento_agenda_lancamento = procura_elemento(driver, "xpath", """//*[@id="EntityDetailsContainer"]/div[1]/div[3]/div[1]/h4/a""", 15)
        if elemento_agenda_lancamento:
            elemento_agenda_lancamento.click()
            if int(mes) == 12:
                mes_agenda = "01"
                ano_agenda = str(int(ano) + 1)
            else:
                if int(mes) > 0 and int(mes) < 10: 
                    mes_agenda = "0" + str(int(mes) + 1)
                elif int(mes) > 9:
                    mes_agenda = str(int(mes) + 1)
                ano_agenda = ano
            today = date.today()
            if today.day > 2 and today.day < 5:
                texto_data_lancamento = f"{str(today.day)}{mes_agenda}{ano_agenda}"
            else:
                texto_data_lancamento = f"02{mes_agenda}{ano_agenda}"
            sleep(2)

            # Vencimento
            elemento_vencimento = encontrar_elemento_shadow_root(driver, "#app", """div > ngb-modal-window > div > div > app-receivement-schedule-create >"""+
                                                                 """ div > div.modal-body > form > div:nth-child(2) > div:nth-child(2) > form-helper:nth-child(1)"""+
                                                                 """ > div:nth-child(1) > div > calendar > div > div > div > input""", 2)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('click', {{'bubbles': true}}));", elemento_vencimento)
            driver.execute_script(f"""arguments[0].value='{texto_data_lancamento}'""", elemento_vencimento)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('input', {{'bubbles': true}}));", elemento_vencimento)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('change', {{'bubbles': true}}));", elemento_vencimento)
            sleep(0.1)

            # Previsão
            elemento_previsao = encontrar_elemento_shadow_root(driver, "#app", """div > ngb-modal-window > div > div > app-receivement-schedule-create > """+
                                                               """div > div.modal-body > form > div:nth-child(2) > div:nth-child(2) > form-helper:nth-child(2)"""+
                                                               """ > div:nth-child(1) > div > calendar > div > div > div > input""", 2)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('click', {{'bubbles': true}}));", elemento_previsao)
            driver.execute_script(f"""arguments[0].value='{texto_data_lancamento}'""", elemento_previsao)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('input', {{'bubbles': true}}));", elemento_previsao)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('change', {{'bubbles': true}}));", elemento_previsao)
            sleep(0.5)
            actions.send_keys(Keys.ESCAPE).perform()
            sleep(0.5)

            # Descrição
            elemento_descricao = encontrar_elemento_shadow_root(driver, "#app", "#description", 2)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('focus', {{'bubbles': true}}));", elemento_descricao)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('click', {{'bubbles': true}}));", elemento_descricao)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('select', {{'bubbles': true}}));", elemento_descricao)
            driver.execute_script("""arguments[0].value='Salários a pagar, FGTS, GPS, provisão direitos trabalhistas, """+
                                  f"""vale transporte e taxa de administração de pessoas {mes}/{ano}'""", elemento_descricao)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('input', {{'bubbles': true}}));", elemento_descricao)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('change', {{'bubbles': true}}));", elemento_descricao)
            sleep(0.1)
            actions.send_keys(Keys.TAB).perform()
            actions.send_keys(Keys.TAB).perform()
            actions.send_keys(Keys.TAB).perform()
            sleep(0.1)

            # Categoria
            elemento_categoria = encontrar_elemento_shadow_root(driver, "#app", """div > ngb-modal-window > div > div > app-receivement-schedule-create > div > div.modal-body > """+ 
                                                                """form > div:nth-child(2) > div.row.mt-3 > app-schedule-category > fieldset > div.ng-untouched.ng-pristine.ng-invalid """+
                                                                """> div > app-schedule-category-item > div > form-helper.col-4 > div:nth-child(1) > div > app-category-select > ng-select """+
                                                                """> div > div > div.ng-input > input[type=text]""", 2)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('focus', {{'bubbles': true}}));", elemento_categoria)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('click', {{'bubbles': true}}));", elemento_categoria)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('select', {{'bubbles': true}}));", elemento_categoria)
            driver.execute_script("""arguments[0].value='Gestão de Mão de Obra Terceirizada'""", elemento_categoria)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('input', {{'bubbles': true}}));", elemento_categoria)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('change', {{'bubbles': true}}));", elemento_categoria)
            sleep(0.1)
            actions.send_keys(Keys.ENTER).perform()
            sleep(0.5)

            # Valor
            elemento_valor = encontrar_elemento_shadow_root(driver, "#app", """div > ngb-modal-window > div > div > app-receivement-schedule-create """+
                                                            """> div > div.modal-body > form > div:nth-child(2) > div.row.mt-3 > app-schedule-category > fieldset > """+
                                                            """div.ng-untouched.ng-invalid.ng-dirty > div > app-schedule-category-item > div > div > form-helper > div:nth-child(1) > div > input""", 2)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('click', {{'bubbles': true}}));", elemento_valor)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('select', {{'bubbles': true}}));", elemento_valor)
            sleep(0.1)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('keydown', {{'bubbles': true}}));", elemento_valor)
            driver.execute_script(f"""arguments[0].value=''""", elemento_valor)
            for i in range(1, len(valor_fatura) + 1):
                substring_atual = valor_fatura[:i]
                driver.execute_script(f"arguments[0].value = '{substring_atual}';", elemento_valor)
                sleep(0.1)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('keypress', {{'bubbles': true}}));", elemento_valor)
            driver.execute_script(f"arguments[0].dispatchEvent(new Event('keyup', {{'bubbles': true}}));", elemento_valor)
            sleep(0.1)

            # Automatizar Cobrança
            elemento_botao_automatizar = encontrar_elemento_shadow_root(driver, "#app", """div > ngb-modal-window > div > div > app-receivement-schedule-create """+
                                                                        """> div > div.modal-body > form > div:nth-child(2) > div:nth-child(7) > panel-toggle >"""+
                                                                        """ div > div > ui-switch > button""", 2)
            driver.execute_script(f"""arguments[0].click();""", elemento_botao_automatizar)
            sleep(0.1)

            # Botao Enviar Imediatamente
            elemento_envio_imediato = encontrar_elemento_shadow_root(driver, "#app", """#entryToday""", 2)
            driver.execute_script(f"""arguments[0].click();""", elemento_envio_imediato)
            sleep(0.1)
            input(f"pera ai, aperte enter para continuar")

            elemento_agendar = encontrar_elemento_shadow_root(driver, "#app", """div > ngb-modal-window > div > div > app-receivement-schedule-create > div > """+
                                                               """div.modal-footer.justify-content-between > div.d-flex.align-items-center.form-check-inline > button""", 10)
            driver.execute_script("""arguments[0].click();""", elemento_agendar)

            sleep(2)
            driver.refresh()
    except Exception as error:
        print(error)


In [17]:
def baixar_boleto_lancamento(driver, valor_fatura, elemento_search, actions):
    print("BAIXANDO BOLETO")
    achou_lancamento = False
    try:
        elemento_lista_lancamentos = procura_todos_elementos(driver, "xpath", """//*[@id="openScheduleList"]/tbody/tr[*]/td[2]/a""", 8)
    except TypeError:
        pass
    try:
        if elemento_lista_lancamentos is not None:
            for elemento in elemento_lista_lancamentos:
                if ("Salários a pagar, FGTS, GPS, provisão direitos trabalhistas, "+
                    f"vale transporte e taxa de administração de pessoas {mes}/{ano}") in elemento.text:
                    achou_lancamento = True
                    elemento.click()
                    sleep(0.5)
                    indice = 1
                    achou_elemento = False
                    while achou_elemento == False:
                        elemento_cobrar_boleto = encontrar_elemento_shadow_root(driver, "#app", f"#ngb-nav-{str(indice)}", 1)
                        if elemento_cobrar_boleto and "Cobrar via boleto" in elemento_cobrar_boleto.text:
                            achou_elemento = True
                            driver.execute_script("""return arguments[0].click();""", elemento_cobrar_boleto)
                            sleep(0.2)
                            elemento_download = encontrar_elemento_shadow_root(driver, "#app", f"""#ngb-nav-{str(indice)}-panel > settings > div > app-schedule-entry-promise > """+
                                                                            """div > app-entry-promise-details-emitted > div > section > table > tbody > tr > """+
                                                                            """td:nth-child(6) > div > a""", 10)
                            sleep(0.2)
                            driver.execute_script("""return arguments[0].click();""", elemento_download)
                        else:
                            indice += 1
                    sleep(1.5)
                    elemento_fechar = encontrar_elemento_shadow_root(driver, "#app", """div > ngb-modal-window > div > div > app-receivement-schedule-details """+
                                                                    """> div > modal-header > div > button""", 10)
                    driver.execute_script("""return arguments[0].click();""", elemento_fechar)
                    sleep(0.5)
                    elemento_search.clear()
                    break
                else:
                    achou_lancamento = False   
            if achou_lancamento == False:
                print("Lançamento do mês não encontrado, fazendo novo agendamento...")
                agendar_lancamento(driver, valor_fatura, actions)
                baixar_boleto_lancamento(driver, valor_fatura, elemento_search, actions)
        else:
            elemento_search.clear()
            print("Nenhum lançamento encontrado!")
    except Exception as error:
        print(f"Deu algum erro ao tentar baixar o boleto: {error}")

In [72]:
def start_chrome():
    try:
        # Iniciar o Chrome com as opções configuradas e o serviço
        print("Iniciando navegador chrome...")
        driver = webdriver.Chrome(service=servico, options=chrome_options) 
        actions = ActionChains(driver)
        driver.maximize_window()
        driver.get("https://passport.nibo.com.br/account/login?email=&returnUrl=%2fauthorize%3fresponse_type"+
                   "%3dtoken%26client_id%3d103416FE-A280-466A-9D28-642ACEE21C3B%26lu%3d1%26redirect_uri%3dhttps"+
                   "%253a%252f%252fempresa.nibo.com.br%252fUser%252fLogonWithToken%253freturnUrl%253d%252fOrganization")
        elemento_email = procura_elemento(driver, "xpath", """//*[@id="Username"]""", 30)
        elemento_email.send_keys(automacao_email)
        elemento_btn_continue = procura_elemento(driver, "xpath","""//*[@id="continue-button"]""", 30)
        elemento_btn_continue.click()
        elemento_senha = procura_elemento(driver, "xpath", """//*[@id="Password"]""", 30)
        elemento_senha.send_keys(automacao_senha)
        elemento_btn_entrar = procura_elemento(driver, "xpath", """//*[@id="password"]/div[3]/input""", 30) 
        elemento_btn_entrar.click()
        return actions, driver
    except Exception as error:
        print(f"Chrome Driver retornou um erro: {error}")
        driver.quit()
        start_chrome()

In [20]:
def envia_arquivos():
    try:  
        input("APERTE QUALQUER TECLA PARA ENVIAR OS ARQUIVOS")
        for diretorio in lista_dir_clientes:
            pastas_regioes = listagem_pastas(diretorio)
            for pasta_cliente in pastas_regioes:
                anexos = []
                extrato = False
                fatura = False
                boleto = False
                nome_pasta_cliente = pega_nome(pasta_cliente)
                sub_pastas_cliente = listagem_pastas(pasta_cliente)
                for sub_pasta in sub_pastas_cliente:
                    if sub_pasta.__contains__(f"{mes}-{ano}"):
                        arquivos_cliente = listagem_arquivos(sub_pasta)
                        for arquivo in arquivos_cliente:
                            if arquivo.__contains__("Extrato_Mensal_") and arquivo.__contains__(f"{nome_pasta_cliente}_{mes}.{ano}.pdf"):
                                extrato = True
                                anexos.append(arquivo)
                            elif arquivo.__contains__("Fatura_Detalhada_") and arquivo.__contains__(f"{nome_pasta_cliente}_{mes}.{ano}.pdf"):
                                fatura = True
                                anexos.append(arquivo)
                            elif arquivo.__contains__("Boleto_Recebimento_") and arquivo.__contains__(f"{nome_pasta_cliente}_{mes}.{ano}.pdf"):
                                boleto = True
                                anexos.append(arquivo)
                        if extrato == True and fatura == True and boleto == True:
                            try:
                                cliente = procura_cliente(nome_pasta_cliente, db_conf)
                                if cliente:
                                    cliente_id = cliente[0]
                                    cliente_email = cliente[4]
                                    valores_extrato = procura_valores(cliente_id, db_conf, mes, ano)
                                    if valores_extrato and valores_extrato[21] == 0 and not cliente_email == None:
                                        print(f"Fará o envio para o cliente {nome_pasta_cliente}")
                                        enviar_email_com_anexos(f"{cliente_email}, {email_gestor}", f"Documentos de Terceirização - {nome_pasta_cliente}", 
                                                               f"{corpo_email}", anexos)
                                        query_atualiza_anexos = ler_sql("sql/atualiza_anexos_cliente.sql")
                                        values_anexos = (cliente_id, mes, ano)
                                        with mysql.connector.connect(**db_conf) as conn, conn.cursor() as cursor:
                                            cursor.execute(query_atualiza_anexos, values_anexos)
                                            conn.commit()
                                        break
                                    elif valores_extrato == None:
                                        print("Valores de financeiro não encontrados!")
                                    elif valores_extrato[21] == 1:
                                        print("Anexos já enviados para o cliente!")
                                    elif cliente_email == None:
                                        print("Cliente sem email!")
                                else:
                                    print("Cliente não encontrado!")
                            except Exception as error:
                                print (error)
                        else:
                            print("Cliente não possui todos os arquivos necessários para o envio!")
    except Exception as error:
        print (error)

In [78]:
if rotina == "3. Gerar Boletos":
    try:
        actions, driver = start_chrome()
        sleep(1)
        elemento_cards = procura_todos_elementos(driver, "xpath", """//*[@id="myorganizations-container"]"""+
                                                """/div/div[3]/ng-include[2]/div[*]/a/h3/span""", 20)
        for card in elemento_cards:
            if card.text == "HUMAN SOLUCOES E DESENVOLVIMENTOS EM RECURSOS HUMANOS LTDA":
                card.click()
                break
        sleep(1.5)
        elemento_contatos = procura_elemento(driver, "xpath", """//*[@id="page-organization-details"]/div[5]/div/div[2]"""+
                                             """/div[2]/div/div/ul[2]/li[3]/a/span""", 20)
        if elemento_contatos:
            actions.move_to_element(elemento_contatos).perform()
            elemento_clientes = procura_elemento(driver, "xpath", """//*[@id="page-organization-details"]/div[5]/div/div[2]"""+
                                                 """/div[2]/div/div/ul[2]/li[3]/ul/li[1]/a/span""", 20)
            elemento_clientes.click()
            sleep(1)
    except Exception as web_error:
        print (web_error)

Iniciando navegador chrome...


In [115]:
try:
    for diretorio in lista_dir_clientes:
        pastas_regioes = listagem_pastas(diretorio)
        for pasta_cliente in pastas_regioes:
            nome_pasta_cliente = pega_nome(pasta_cliente)
            sub_pastas_cliente = listagem_pastas(pasta_cliente)
            for sub_pasta in sub_pastas_cliente:
                if sub_pasta.__contains__(f"{mes}-{ano}"):
                    caminho_destino = Path(sub_pasta)
                    arquivos_cliente = listagem_arquivos(sub_pasta)
                    for arquivo in arquivos_cliente:
                        if arquivo.__contains__("Boleto_Recebimento_") and arquivo.__contains__(nome_pasta_cliente):
                            boleto = True
                            break
                        else:
                            boleto = False
                    if boleto == True:
                        boleto = False
                        break
                    elif boleto == False:
                        cliente = procura_cliente(nome_pasta_cliente, db_conf)
                        if cliente:
                            cliente_id = cliente[0]
                            cliente_cnpj = cliente[2]
                            cliente_cpf = cliente[3]
                            valores = procura_valores(cliente_id, db_conf, mes, ano)
                            if valores:
                                valor_fatura = valores[20]
                                valor_fatura_formatado = f"{valor_fatura:.2f}".replace(".", ",")
                                print(f"{nome_pasta_cliente} vai precisar de um boleto. Valor da fatura é: {valor_fatura}")
                                elemento_search = procura_elemento(driver, "xpath", """//*[@id="entityList_filter"]"""+
                                                                    """/label/input""", 15)   
                                if elemento_search:                           
                                    if not cliente_cnpj == '' or not cliente_cnpj == None:
                                        elemento_search.send_keys(cliente_cnpj)
                                    elif not cliente_cpf == '' or not cliente_cpf == None:
                                        elemento_search.send_keys(cliente_cpf)                       
                                sleep(2)
                                try:
                                    elemento_lista_clientes = procura_todos_elementos(driver, "xpath", """//*[@id="entityList"]"""+
                                                                """/tbody/tr/td[1]/a""" , 15)
                                except NoSuchElementException:
                                    elemento_lista_clientes = procura_todos_elementos(driver, "xpath", """//*[@id="entityList"]"""+
                                                                """/tbody/tr[*]/td[1]/a""" , 15)
                                for cliente_lista in elemento_lista_clientes:
                                    if cliente_lista.text.__contains__(cliente_cnpj) or cliente_lista.text.__contains__(cliente_cpf):
                                        cliente_lista.click()
                                        sleep(1)
                                        try:
                                            sleep(0.7)
                                            elemento_sem_lancamento = procura_elemento(driver, "class_name", """generic-list-no-content""", 4)
                                            if elemento_sem_lancamento:
                                                agendar_lancamento(driver, valor_fatura_formatado, actions)
                                                sleep(1.5)
                                                baixar_boleto_lancamento(driver, valor_fatura_formatado, elemento_search, actions)
                                            elif elemento_sem_lancamento == None:
                                                baixar_boleto_lancamento(driver, valor_fatura_formatado, elemento_search, actions)                                                   
                                            sleep(4)
                                            arquivos_downloads = listagem_arquivos_downloads()
                                            arquivo_mais_recente = max(arquivos_downloads, key=os.path.getmtime)
                                            if (arquivo_mais_recente.__contains__(".pdf") 
                                                and not arquivo_mais_recente.__contains__(f"Boleto_Recebimento_{nome_pasta_cliente.replace("S/S", "S S")}_{mes}.{ano}")):
                                                caminho_pdf = Path(arquivo_mais_recente)
                                                novo_nome_boleto = caminho_pdf.with_name(f"Boleto_Recebimento_{nome_pasta_cliente.replace("S/S", "S S")}_{mes}.{ano}.pdf")
                                                caminho_pdf_mod = caminho_pdf.rename(novo_nome_boleto)
                                                sleep(0.5)
                                                copy(caminho_pdf_mod, caminho_destino / caminho_pdf_mod.name)
                                                if os.path.exists(caminho_pdf_mod):
                                                    os.remove(caminho_pdf_mod)
                                        except NoSuchElementException:
                                            print("Algum objeto nao foi encontrado!")
                            else:
                                print(f"Valores de financeiro não encontrados para {nome_pasta_cliente}")
                        else:
                            print(f"Cliente {nome_pasta_cliente} não encontrado!")                    
except Exception as error:
    print(error)

CODER INSIGHTS DESENVOLVIMENTO DE SOFTWARE LTDA vai precisar de um boleto. Valor da fatura é: 800.0
AGENDANDO LANÇAMENTO


KeyboardInterrupt: Interrupted by user